---
## Setup

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

from src.step_tracker import GBFSStepTracker, BPSOStepTracker, DPStepTracker
from src.step_visualizer import StepByStepVisualizer
from src.test_case_loader import TestCaseLoader

print("✅ Setup complete")

✅ Setup complete


---
# PHẦN 1: GBFS STEP-BY-STEP

## 1.1. Chạy GBFS với tracking

In [2]:
# Load small test case for easy visualization
loader = TestCaseLoader()
test_case = loader.load_test_case('Size Small 30')

print(f"📦 Test Case: {test_case['test_case_name']}")
print(f"   Items: {len(test_case['items'])}")
print(f"   Capacity: {test_case['capacity']}\n")

# Run GBFS with tracking
gbfs_tracker = GBFSStepTracker()
result = gbfs_tracker.solve_with_tracking(
    test_case['items'],
    test_case['weights'],
    test_case['values'],
    test_case['capacity'],
    max_states=1000
)

solution = result['solution']
tracker = result['tracker']

print(f"✅ Solution found:")
print(f"   Value: {solution['total_value']:.1f}")
print(f"   Weight: {solution['total_weight']:.1f} / {test_case['capacity']}")
print(f"   Items: {len(solution['selected_items'])}")
print(f"   Steps recorded: {tracker.get_total_steps()}")

FileNotFoundError: ⚠️  Summary file not found at data/test_cases/test_cases_summary.csv
Please run: python src/data_generator.py

## 1.2. Interactive Step Viewer

In [3]:
# Create visualizer
visualizer = StepByStepVisualizer(figsize=(16, 10))

# Current step tracker
current_step = [0]
total_steps = tracker.get_total_steps()

# Create output widget
output = widgets.Output()

def show_step(step_index):
    """Display step visualization"""
    with output:
        clear_output(wait=True)
        
        if 0 <= step_index < total_steps:
            step_data = tracker.get_step(step_index)
            fig = visualizer.visualize_gbfs_step(
                step_data,
                test_case['items'],
                np.array(test_case['weights']),
                np.array(test_case['values'])
            )
            plt.show()
        else:
            print(f"Invalid step: {step_index}")

# Control buttons
def on_previous_clicked(b):
    current_step[0] = max(0, current_step[0] - 1)
    step_slider.value = current_step[0]

def on_next_clicked(b):
    current_step[0] = min(total_steps - 1, current_step[0] + 1)
    step_slider.value = current_step[0]

def on_slider_change(change):
    current_step[0] = change['new']
    show_step(current_step[0])

# Create widgets
prev_button = widgets.Button(description="◀ Previous", button_style='info')
next_button = widgets.Button(description="Next ▶", button_style='info')
step_slider = widgets.IntSlider(
    value=0, min=0, max=total_steps-1,
    description='Step:', continuous_update=False
)

prev_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)
step_slider.observe(on_slider_change, names='value')

# Layout
controls = widgets.HBox([prev_button, step_slider, next_button])
display(controls)
display(output)

# Show first step
show_step(0)

NameError: name 'tracker' is not defined

## 1.3. Phân tích các bước quan trọng

In [4]:
# Find key steps
print("📊 Key Steps Analysis:\n")

for i in range(total_steps):
    step = tracker.get_step(i)
    if step['type'] in ['start', 'select', 'goal']:
        print(f"Step {i}: {step['type'].upper()}")
        print(f"  {step['message']}")
        print(f"  Value: {step['value']:.1f}, Weight: {step['weight']:.1f}")
        print()

📊 Key Steps Analysis:



NameError: name 'total_steps' is not defined

---
# PHẦN 2: BPSO STEP-BY-STEP

## 2.1. Chạy BPSO với tracking

In [5]:
# Run BPSO with tracking
bpso_tracker = BPSOStepTracker()
result_bpso = bpso_tracker.solve_with_tracking(
    test_case['items'],
    test_case['weights'],
    test_case['values'],
    test_case['capacity'],
    n_particles=20,
    max_iterations=50
)

solution_bpso = result_bpso['solution']
tracker_bpso = result_bpso['tracker']

print(f"✅ BPSO Solution:")
print(f"   Value: {solution_bpso['total_value']:.1f}")
print(f"   Weight: {solution_bpso['total_weight']:.1f}")
print(f"   Items: {len(solution_bpso['selected_items'])}")
print(f"   Iterations tracked: {tracker_bpso.get_total_steps()}")

NameError: name 'test_case' is not defined

## 2.2. Interactive BPSO Viewer

In [6]:
# Create BPSO visualizer
visualizer_bpso = StepByStepVisualizer(figsize=(16, 10))

current_iter = [0]
total_iters = tracker_bpso.get_total_steps()

output_bpso = widgets.Output()

def show_bpso_step(iter_index):
    with output_bpso:
        clear_output(wait=True)
        
        if 0 <= iter_index < total_iters:
            step_data = tracker_bpso.get_step(iter_index)
            fig = visualizer_bpso.visualize_bpso_step(
                step_data,
                test_case['items'],
                np.array(test_case['weights']),
                np.array(test_case['values']),
                test_case['capacity']
            )
            plt.show()

def on_bpso_slider_change(change):
    current_iter[0] = change['new']
    show_bpso_step(current_iter[0])

iter_slider = widgets.IntSlider(
    value=0, min=0, max=total_iters-1,
    description='Iteration:', continuous_update=False
)

iter_slider.observe(on_bpso_slider_change, names='value')

display(iter_slider)
display(output_bpso)

show_bpso_step(0)

NameError: name 'tracker_bpso' is not defined

## 2.3. Convergence Analysis

In [7]:
# Extract convergence data
iterations = []
best_fitness = []
avg_fitness = []

for i in range(total_iters):
    step = tracker_bpso.get_step(i)
    iterations.append(step['iteration'])
    best_fitness.append(step.get('gbest_fitness', 0))
    avg_fitness.append(step.get('avg_fitness', 0))

# Plot convergence
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(iterations, best_fitness, 'b-', linewidth=2, label='Best Fitness')
ax.plot(iterations, avg_fitness, 'g--', linewidth=2, label='Average Fitness')
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Fitness', fontsize=12)
ax.set_title('BPSO Convergence Curve', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Statistics
print(f"\n📊 Convergence Statistics:")
print(f"   Initial Best: {best_fitness[0]:.1f}")
print(f"   Final Best: {best_fitness[-1]:.1f}")
print(f"   Improvement: {best_fitness[-1] - best_fitness[0]:.1f} ({((best_fitness[-1]/best_fitness[0]-1)*100):.1f}%)")
print(f"   Converged at iteration: {next((i for i, v in enumerate(best_fitness) if v == best_fitness[-1]), -1)}")

NameError: name 'total_iters' is not defined

---
# PHẦN 3: SO SÁNH CÁCH THUẬT TOÁN HOẠT ĐỘNG

## 3.1. Insights

In [8]:
print("🔍 Algorithm Behavior Analysis:\n")

print("GBFS (Greedy Best-First Search):")
print("  • Chiến lược: Luôn chọn item có ratio value/weight cao nhất")
print("  • Ưu điểm: Nhanh, deterministic, dễ hiểu")
print("  • Nhược điểm: Có thể bỏ lỡ global optimum")
print(f"  • Steps: {tracker.get_total_steps()}")
print(f"  • Result: {solution['total_value']:.1f}\n")

print("BPSO (Binary Particle Swarm Optimization):")
print("  • Chiến lược: Particles khám phá không gian tìm kiếm")
print("  • Ưu điểm: Exploration tốt, không cần heuristic")
print("  • Nhược điểm: Stochastic, nhiều tham số")
print(f"  • Iterations: {total_iters}")
print(f"  • Result: {solution_bpso['total_value']:.1f}\n")

print("Observations:")
print(f"  • GBFS took {tracker.get_total_steps()} steps")
print(f"  • BPSO took {total_iters} iterations with {20} particles each")
print(f"  • Value difference: {abs(solution['total_value'] - solution_bpso['total_value']):.1f}")

🔍 Algorithm Behavior Analysis:

GBFS (Greedy Best-First Search):
  • Chiến lược: Luôn chọn item có ratio value/weight cao nhất
  • Ưu điểm: Nhanh, deterministic, dễ hiểu
  • Nhược điểm: Có thể bỏ lỡ global optimum


NameError: name 'tracker' is not defined

---
# KẾT LUẬN

## Điều học được:

### GBFS:
- Mỗi bước chọn item tốt nhất theo heuristic
- Quá trình tuyến tính, dễ theo dõi
- Có thể thấy tại sao chọn item đó (ratio cao nhất)

### BPSO:
- Particles khám phá song song
- Convergence theo thời gian
- Balance giữa exploration (tìm kiếm mới) và exploitation (tối ưu local)

## Ứng dụng:
- Step-by-step giúp **debug** thuật toán
- Giúp **hiểu** tại sao thuật toán chọn solution đó
- Hữu ích cho **giảng dạy** và **demo**